In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import xml.etree.ElementTree as ET
import re
import nltk
nltk.download('stopwords')

tree = ET.parse("/content/drive/MyDrive/datasets/NLP/corpus_taln_v1.tei.xml") #L'XML du corpus taln est à insérer ici.
root = tree.getroot()
ns = re.findall("{.*}", root.tag)[0] #Le nom de domaine XML du corpus.
ns 
# {http://www.tei-c.org/ns/1.0}

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


'{http://www.tei-c.org/ns/1.0}'

Dans la fonction get_abstracts j'utilise la librairie xml elementtree comme conseillé dans le sujet du projet 1 pour :

1.   itérer sur les balises des articles "TEI" qui contiennent dans leur attribut de valise le mot clé "fr" qui signifie que l'article est en français,

2.   naviguer à la balise div de l'entête de l'article qui contient les résumés dont je récupère la version en français si elle existe et est différente de "None".

In [3]:
def Get_Abstracts(tree, size=None):
  abstract = []
  text = ''
  tei = root.iter(ns+"TEI") #Itérateur sur les balises 'TEI' qui correspondent aux articles.
  if size == None:
    size = root.iter(ns+"TEI")
  else:
    size = range(size)
  for i in size:
    try:
      x = tei.__next__()
      if any("fr" in s for s in x.attrib.values()):
        for y in x.iter(ns+"div"):
          if any("fr" in s for s in y.attrib.values()):
            for p in y.iter(ns+"p"):
              if p.text != "None":
                text = p.text.strip().replace('\n\t','') #Nettoyage de ce qui déborde dans les abstracts avec la fonction strip() et effacement des caractères de saut à la ligne présent dans la chaine de caractère brute.
        if len(text) != 0:
          abstract.append(text)
    except:
      break
  return abstract

In [44]:
abstracts = Get_Abstracts(root)
abstracts[0:5]

['Nous considérons dans notre travail la tâche du traitement automatique visant à construire, à partir de textes issus d\'un corpus de constats d\'accidents de la route, des interprétations compatibles avec ces derniers, et à en proposer des illustrations sous forme de séquences d\'images fixes. Notre recherche est le fruit d\'une collaboration entre un laboratoire universitaire et une entreprise. Elle prend appui sur le modèle de la Grammaire Applicative et Cognitive [DES 90], qui vise en particulier à "expliquer", à un certain niveau cognitif, les transferts entre représentations imagées et verbales. Pour une revue de la question relative à la "transcription automatique Verbal-Image", nous renvoyons à [ARN 90] ; et plus particulièrement aux travaux de C. Vandeloise [VAN 87] et du groupe "Langue, Raisonnement, Calcul" de l\'Université Paul Sabatier [AUR 90, SAB 95] ainsi qu\'aux approches proposées dans [ARN 93] et dans le système SPRINT [YAM 92]. Plus proches encore de nos préoccupat

J'utilise la librairie disponible à ce [lien](https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer) qui permet la lémmatisation des mots français en se basant sur des ressources académiques du [Lexique des Formes Fléchies du Français](http://pauillac.inria.fr/~sagot/index.html#lefff).


In [5]:
!pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git

  Cloning https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git to /tmp/pip-req-build-6amagics
  Running command git clone -q https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git /tmp/pip-req-build-6amagics
  Created wheel for FrenchLefffLemmatizer: filename=FrenchLefffLemmatizer-0.3-cp36-none-any.whl size=3533520 sha256=8ebef7656ab2fd981ad3f26236a959ddcaff9e17b98157607361fa92f5923bcf
  Stored in directory: /tmp/pip-ephem-wheel-cache-wkxmwa3o/wheels/95/b7/c0/e249ca2690c04f6106b9581c5e4111287f71dbd85bac903445
Successfully built FrenchLefffLemmatizer


Les abstracts doivent être nettoyer et épurer des mots d'arrêt basique comme prétraitement avant de faire le tf-idf, je procède à ces étapes :


1.   Séparation des résumés en token avec la fonction regex de tokénisation de nltk,

2.   Utilisisation de la liste de mots d'arrêt de meilleure qualité que celle de nltk trouvé [ici](https://github.com/cmchurch/nltk_french/blob/master/french-nltk.py) dans la fonction "get_stopswords",

3.   Comme en travaux pratiques dirigés, le texte est mis en miniscule avec .lower(), a noté que la double boucle est dû à des précédentes méthodes que j'ai utilisé et qui ont donné la forme peut pratique d'une liste de liste, ainsi il est nécessaire pour accéder aux chaines de caractères de faire deux fois des for x ~ in y

4.   J'utilise ensuite la fonctionnalité de python sur la génération de liste avec opération sur chaque élement pour retirer les mots d'arrêt simples puis tous les nombres et mots courts de trois lettres ou moins,

5. Pour finir j'utilise la librairie référencer plus haut pour lemmatiser la liste de mots d'arrêt.



In [6]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from gensim import corpora, models
import re

def clean_text(text):
  lemmatizer = FrenchLefffLemmatizer()
  tokenizer = RegexpTokenizer(r'\w+')
  fr_stop = ["Ap.", "Apr.", "GHz", "MHz", "USD", "a", "afin", "ah", "ai", "aie", "aient", "aies", "ait", "alors", "après", "as", "attendu", "au", "au-delà", "au-devant", "aucun", "aucune", "audit", "auprès", "auquel", "aura", "aurai", "auraient", "aurais", "aurait", "auras", "aurez", "auriez", "aurions", "aurons", "auront", "aussi", "autour", "autre", "autres", "autrui", "aux", "auxdites", "auxdits", "auxquelles", "auxquels", "avaient", "avais", "avait", "avant", "avec", "avez", "aviez", "avions", "avons", "ayant", "ayez", "ayons", "b", "bah", "banco", "ben", "bien", "bé", "c", "c'", "c'est", "c'était", "car", "ce", "ceci", "cela", "celle", "celle-ci", "celle-là", "celles", "celles-ci", "celles-là", "celui", "celui-ci", "celui-là", "celà", "cent", "cents", "cependant", "certain", "certaine", "certaines", "certains", "ces", "cet", "cette", "ceux", "ceux-ci", "ceux-là", "cf.", "cg", "cgr", "chacun", "chacune", "chaque", "chez", "ci", "cinq", "cinquante", "cinquante-cinq", "cinquante-deux", "cinquante-et-un", "cinquante-huit", "cinquante-neuf", "cinquante-quatre", "cinquante-sept", "cinquante-six", "cinquante-trois", "cl", "cm", "cm²", "comme", "contre", "d", "d'", "d'après", "d'un", "d'une", "dans", "de", "depuis", "derrière", "des", "desdites", "desdits", "desquelles", "desquels", "deux", "devant", "devers", "dg", "différentes", "différents", "divers", "diverses", "dix", "dix-huit", "dix-neuf", "dix-sept", "dl", "dm", "donc", "dont", "douze", "du", "dudit", "duquel", "durant", "dès", "déjà", "e", "eh", "elle", "elles", "en", "en-dehors", "encore", "enfin", "entre", "envers", "es", "est", "et", "eu", "eue", "eues", "euh", "eurent", "eus", "eusse", "eussent", "eusses", "eussiez", "eussions", "eut", "eux", "eûmes", "eût", "eûtes", "f", "fait", "fi", "flac", "fors", "furent", "fus", "fusse", "fussent", "fusses", "fussiez", "fussions", "fut", "fûmes", "fût", "fûtes", "g", "gr", "h", "ha", "han", "hein", "hem", "heu", "hg", "hl", "hm", "hm³", "holà", "hop", "hormis", "hors", "huit", "hum", "hé", "i", "ici", "il", "ils", "j", "j'", "j'ai", "j'avais", "j'étais", "jamais", "je", "jusqu'", "jusqu'au", "jusqu'aux", "jusqu'à", "jusque", "k", "kg", "km", "km²", "l", "l'", "l'autre", "l'on", "l'un", "l'une", "la", "laquelle", "le", "lequel", "les", "lesquelles", "lesquels", "leur", "leurs", "lez", "lors", "lorsqu'", "lorsque", "lui", "lès", "m", "m'", "ma", "maint", "mainte", "maintes", "maints", "mais", "malgré", "me", "mes", "mg", "mgr", "mil", "mille", "milliards", "millions", "ml", "mm", "mm²", "moi", "moins", "mon", "moyennant", "mt", "m²", "m³", "même", "mêmes", "n", "n'avait", "n'y", "ne", "neuf", "ni", "non", "nonante", "nonobstant", "nos", "notre", "nous", "nul", "nulle", "nº", "néanmoins", "o", "octante", "oh", "on", "ont", "onze", "or", "ou", "outre", "où", "p", "par", "par-delà", "parbleu", "parce", "parmi", "pas", "passé", "pendant", "personne", "peu", "plus", "plus_d'un", "plus_d'une", "plusieurs", "pour", "pourquoi", "pourtant", "pourvu", "près", "puisqu'", "puisque", "q", "qu", "qu'", "qu'elle", "qu'elles", "qu'il", "qu'ils", "qu'on", "quand", "quant", "quarante", "quarante-cinq", "quarante-deux", "quarante-et-un", "quarante-huit", "quarante-neuf", "quarante-quatre", "quarante-sept", "quarante-six", "quarante-trois", "quatorze", "quatre", "quatre-vingt", "quatre-vingt-cinq", "quatre-vingt-deux", "quatre-vingt-dix", "quatre-vingt-dix-huit", "quatre-vingt-dix-neuf", "quatre-vingt-dix-sept", "quatre-vingt-douze", "quatre-vingt-huit", "quatre-vingt-neuf", "quatre-vingt-onze", "quatre-vingt-quatorze", "quatre-vingt-quatre", "quatre-vingt-quinze", "quatre-vingt-seize", "quatre-vingt-sept", "quatre-vingt-six", "quatre-vingt-treize", "quatre-vingt-trois", "quatre-vingt-un", "quatre-vingt-une", "quatre-vingts", "que", "quel", "quelle", "quelles", "quelqu'", "quelqu'un", "quelqu'une", "quelque", "quelques", "quelques-unes", "quelques-uns", "quels", "qui", "quiconque", "quinze", "quoi", "quoiqu'", "quoique", "r", "revoici", "revoilà", "rien", "s", "s'", "sa", "sans", "sauf", "se", "seize", "selon", "sept", "septante", "sera", "serai", "seraient", "serais", "serait", "seras", "serez", "seriez", "serions", "serons", "seront", "ses", "si", "sinon", "six", "soi", "soient", "sois", "soit", "soixante", "soixante-cinq", "soixante-deux", "soixante-dix", "soixante-dix-huit", "soixante-dix-neuf", "soixante-dix-sept", "soixante-douze", "soixante-et-onze", "soixante-et-un", "soixante-et-une", "soixante-huit", "soixante-neuf", "soixante-quatorze", "soixante-quatre", "soixante-quinze", "soixante-seize", "soixante-sept", "soixante-six", "soixante-treize", "soixante-trois", "sommes", "son", "sont", "sous", "soyez", "soyons", "suis", "suite", "sur", "sus", "t", "t'", "ta", "tacatac", "tandis", "te", "tel", "telle", "telles", "tels", "tes", "toi", "ton", "toujours", "tous", "tout", "toute", "toutefois", "toutes", "treize", "trente", "trente-cinq", "trente-deux", "trente-et-un", "trente-huit", "trente-neuf", "trente-quatre", "trente-sept", "trente-six", "trente-trois", "trois", "très", "tu", "u", "un", "une", "unes", "uns", "v", "vers", "via", "vingt", "vingt-cinq", "vingt-deux", "vingt-huit", "vingt-neuf", "vingt-quatre", "vingt-sept", "vingt-six", "vingt-trois", "vis-à-vis", "voici", "voilà", "vos", "votre", "vous", "w", "x", "y", "z", "zéro", "à", "ç'", "ça", "ès", "étaient", "étais", "était", "étant", "étiez", "étions", "été", "étée", "étées", "étés", "êtes", "être", "ô"]
  cleaned_text = []
  for abstract in text:
      lowercase_text = abstract.lower()
      words = tokenizer.tokenize(lowercase_text)
      non_stopped_words = [lemmatizer.lemmatize(i) for i in words if not i in fr_stop and not bool(re.search("\d+",i)) and len(i)>3]
      cleaned_text.append(non_stopped_words)
  return cleaned_text
  
def doLDA(cleaned_text):
  dictionary = corpora.Dictionary(cleaned_text)
  corpus = [dictionary.doc2bow(abstract) for abstract in cleaned_text]
  ldamodel = models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary)
  print(ldamodel.print_topics(num_topics=2, num_words=4))
  return ldamodel

Comme dans les exemples trouvables sur internet, j'ai utilisé l'implémentation de tfidf de sklearn ce qui me permet de récupérer le résulat final soit forme d'un Dataframe pandas avec en abscisse les mots d'arrêt et en ordonné l'index des résumés.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

text = clean_text(text = abstracts)
corpus = [' '.join(doc) for doc in text]
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df

,abandonnant,abandonné,able,abord,abordant,abrupte,abrégé,absence,absent,abstractif,abstractive,abstraite,abstraites,académiques,accent,accepte,acception,acceptées,accessibles,accident,accompagne,accord,accordé,accès,accéder,acoustique,acquisition,acquisitionniste,acquière,actancielle,actant,acte,actif,action,activation,active,activité,actuelle,actuellement,acyclic,...,étiquetagemorpho,étiqueteur,étiquettage,étiquette,étiqueté,étiquetés,étranger,étude,étudiant,étudier,étudions,étudié,étudiées,étudiés,étymologie,évaluatif,évaluation,évaluative,évaluatives,évaluer,évaluerons,évaluée,éventail,éventualité,évidence,évident,éviter,évolutif,évolutifs,évolution,évolutive,évolutives,évolutivité,évoque,évoquées,évènementielle,évènements,événement,événementielle,événementiels
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.082459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.078073,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.145592,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

Les mots ayant un tfidf de 0 ne sont pas présent dans les résumés des lignes correspondantes, ils ne sont donc pas pertinent, je crée donc un deuxième jeu de donnée fait des valeurs strictement positives.

La fonction describe donne des statistiques sur le jeu de donnée, celle ci est utile pour connaitre la répartition des valeurs td idf pertinentes.

In [8]:
desc = df[df > 0].describe()
desc

,abandonnant,abandonné,able,abord,abordant,abrupte,abrégé,absence,absent,abstractif,abstractive,abstraite,abstraites,académiques,accent,accepte,acception,acceptées,accessibles,accident,accompagne,accord,accordé,accès,accéder,acoustique,acquisition,acquisitionniste,acquière,actancielle,actant,acte,actif,action,activation,active,activité,actuelle,actuellement,acyclic,...,étiquetagemorpho,étiqueteur,étiquettage,étiquette,étiqueté,étiquetés,étranger,étude,étudiant,étudier,étudions,étudié,étudiées,étudiés,étymologie,évaluatif,évaluation,évaluative,évaluatives,évaluer,évaluerons,évaluée,éventail,éventualité,évidence,évident,éviter,évolutif,évolutifs,évolution,évolutive,évolutives,évolutivité,évoque,évoquées,évènementielle,évènements,événement,événementielle,événementiels
count,1.000000,1.000000,1.00000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000,1.000000,1.000000,5.000000,1.000000,1.000000,1.000000,1.000000,6.000000,1.000000,4.000000,1.000000,3.000000,28.000000,2.000000,1.00000,2.000000,1.000000,12.000000,3.000000,1.00000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,...,1.000000,8.000000,1.000000,3.000000,5.000000,2.000000,2.000000,15.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,85.000000,1.000000,1.000000,4.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000,2.000000,1.000000,1.000000,3.000000,1.00000,2.000000,1.000000,1.000000,1.00000,1.000000,1.000000,10.000000,1.000000,1.000000
mean,0.082296,0.096364,0.49014,0.108237,0.153754,0.108755,0.098484,0.199494,0.042082,0.383112,0.361251,0.380473,0.342702,0.078354,0.082296,0.042082,0.279619,0.138659,0.096389,0.082459,0.078354,0.376711,0.132849,0.385082,0.180725,0.381658,0.341043,0.145834,0.16775,0.412000,0.476413,0.296847,0.417863,0.23954,0.477939,0.448518,0.272315,0.146578,0.179809,0.271434,...,0.533672,0.360660,0.661729,0.410997,0.318679,0.229877,0.394629,0.243023,0.078354,0.144373,0.187268,0.114487,0.180725,0.193035,0.562541,0.608532,0.308624,0.458665,0.321748,0.105788,0.134081,0.172197,0.152857,0.411316,0.089011,0.078354,0.109820,0.344205,0.184644,0.254174,0.54228,0.298336,0.511433,0.163373,0.22801,0.411759,0.607883,0.440063,0.594895,0.524093
std,NaN,NaN,NaN,0.023640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.156819,NaN,NaN,NaN,0.184360,NaN,NaN,NaN,NaN,0.163013,NaN,0.179554,NaN,0.038002,0.104128,0.000000,NaN,0.055251,NaN,0.133793,0.067838,NaN,NaN,NaN,0.189427,0.032545,NaN,NaN,...,NaN,0.207922,NaN,0.079683,0.232430,0.212400,0.096411,0.136131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.120218,NaN,NaN,0.058638,NaN,NaN,NaN,NaN,0.044391,NaN,0.044898,NaN,NaN,0.102108,NaN,0.174677,NaN,NaN,NaN,NaN,NaN,0.052591,NaN,NaN
min,0.082296,0.096364,0.49014,0.070434,0.153754,0.108755,0.098484,0.199494,0.042082,0.383112,0.361251,0.380473,0.145592,0.078354,0.082296,0.042082,0.036016,0.138659,0.096389,0.082459,0.078354,0.098664,0.132849,0.118352,0.180725,0.351925,0.108891,0.145834,0.16775,0.372931,0.476413,0.062085,0.360501,0.23954,0.477939,0.448518,0.138369,0.123565,0.179809,0.271434,...,0.533672,0.096134,0.661729,0.319018,0.036016,0.079687,0.326456,0.058839,0.078354,0.144373,0.187268,0.114487,0.180725,0.193035,0.562541,0.608532,0.055999,0.458665,0.321748,0.037023,0.134081,0.172197,0.152857,0.411316,0.037023,0.078354,0.078073,0.344205,0.184644,0.170236,0.54228,0.174820,0.511433,0.163373,0.22801,0.411759,0.607883,0.376107,0.594895,0.524093
25%,0.082296,0.096364,0.49014,0.102025,0.153754,0.108755,0.098484,0.199494,0.042082,0.383112,0.361251,0.380473,0.275764,0.078354,0.082296,0.042082,0.131590,0.138659,0.096389,0.082459,0.078354,0.310124,0.132849,0.359586,0.180725,0.360251,0.250554,0.145834,0.16775,0.392465,0.476413,0.230344,0.380424,0.23954,0.477939,0.448518,0.205342,0.135071,0.179809,0.271434,...,0.533672,0.243898,0.661729,0.386965,0.114754,0.154782,0.360542,0.091839,0.078354,0.144373,0.187268,0.114487,0.180725,0.193035,0.562541,0.608532,0.245567,0.458665,0.321748,0.072856,0.134081,0.172197,0.152857,0.411316,0.060956

J'extraie la ligne des moyennes que je place dans une variable sous forme d'un tableau trié de manière descendante.

In [9]:
tf_idf_mean = desc.iloc[1,:]
mean_of_means = tf_idf_mean.sort_values(ascending=False).values
mean_of_means

array([0.88552986, 0.87503543, 0.85792386, ..., 0.04208241, 0.04208241,
       0.04208241])

La coupe se fait sur l'index de la partie entière du quart de la longueur du tableau, la valeur ext extraite dans une variable.

In [10]:
mean_of_means = mean_of_means[int(len(mean_of_means)*1/4)]
mean_of_means

0.4300170482020056

Les mots finaux sont ceux qui ont un score tfidf supérieur ou égale à la valeur discriminatoire déterminer plus tôt.

In [11]:
words = tf_idf_mean[tf_idf_mean >= mean_of_means].index
words = pd.DataFrame(words)
words = words.rename(columns={0:"words"})
words

,words
0,able
1,actant
2,activation
3,active
4,adapatabilité
...,...
746,évolutivité
747,évènements
748,événement
749,événementielle


J'épure le texte des mots d'arrêts déterminés par tfidf.

In [43]:
final_text = [i for i in text if i in words.values]
final_text[0:4]

[['déplacer',
  'pied',
  'moyen',
  'activité',
  'humaine',
  'courante',
  'trouver',
  'chemin',
  'suppose',
  'souvent',
  'aide',
  'type',
  'verbal',
  'description',
  'itinéraire',
  'iconique',
  'croquis',
  'carte',
  'présentons',
  'article',
  'système',
  'capable',
  'produire',
  'description',
  'itinéraire',
  'métro',
  'générateur',
  'fondé',
  'cognitif',
  'production',
  'description',
  'analyse',
  'corpus',
  'montrons',
  'comment',
  'facteur',
  'importance',
  'relative',
  'information',
  'choix',
  'stylistique',
  'peuvent',
  'faire',
  'varier',
  'foi',
  'contenu',
  'forme',
  'description',
  'itinéraire'],
 ['expression',
  'spatio',
  'temporalité',
  'traditionnellement',
  'scindée',
  'paradigme',
  'localisation',
  'déplacement',
  'localisation',
  'exprime',
  'nombre',
  'relation',
  'entité',
  'localiser',
  'site',
  'déplacement',
  'exprime',
  'changement',
  'relation',
  'temps',
  'omettre',
  'autonomie',
  'richesse',
 

In [ ]:
doLDA(words.values)

[(0, '0.002*"dicovalence" + 0.002*"répétés" + 0.002*"empiriques" + 0.002*"jeu"'), (1, '0.002*"négation" + 0.002*"world" + 0.002*"extension" + 0.002*"tourisme"')]


Transformers est une librairie qui utilise huggingface dans un environnement pytorch et tensorflow.

In [13]:
!pip install transformers

     |████████████████████████████████| 1.4MB 12.4MB/s 
     |████████████████████████████████| 890kB 43.5MB/s 
     |████████████████████████████████| 2.9MB 53.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=fa8bb40f6fe39ec1b054ab2f2c4f2fbcf96ef36f59afdce5564b7e061e7e9f89
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


T5-base est un modèle encodeur décodeur entrainé sur des articles de presse,  il est capable de faire des résumés en prédisant à partir d'une séquence de représentation vectorielle d'une chaine de caractère une autre chaine plus petite. Sa capacité d'entrée est limité à 512 vecteurs.

In [16]:
from transformers import AutoModelWithLMHead, AutoTokenizer

model = AutoModelWithLMHead.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:852: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Avec cette fonction je transforme ma liste de résumé d'article du corpus en liste des résumés concaténés s'ils font moins de 512 mots ensemble.
Ceux qui font plus de 512 mots sont ajoutés à la liste immédiatement, ils seront tronqué par le modèle.

In [17]:
def Make_Data_For_T5(text_list,token_func):
  text_for_model = []
  holder = ''
  for i in range(len(text_list)):
    if len(token_func.encode(text_list[i]))>512:
      text_for_model.append(text_list[i])
    else:
      if len(token_func.encode(holder))+len(token_func.encode(text_list[i]))>512:
        text_for_model.append(holder)
        holder = ''
      else:
        holder += text_list[i]
  return text_for_model
text_for_model = Make_Data_For_T5(abstracts,tokenizer)

In [18]:
len(text_for_model)

94

In [19]:
summaries = []
for i in text_for_model:
  inputs = tokenizer.encode(i, return_tensors="pt", max_length=512)
  outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
  summaries.append(tokenizer.decode(outputs[0]))

In [38]:
summaries_for_model = Make_Data_For_T5(summaries,tokenizer)
summaries_for_model = [re.sub("<pad>","",i) for i in summaries_for_model]
summaries_for_model = [re.sub("<extra_id_\d>","",i) for i in summaries_for_model]
summaries_for_model[0:4]

[' à en proposer des illustrations sous forme de séquences fixes. Notre recherche est le fruit d\'une collaboration entre un laboratoire universitaire et une entreprise. Elle prend appui sur le modèle de la Grammaire Applicative et Cognitive [DES 90] qui vise à "expliquer", à un certain niveau cognitif, les transferts entre représentations imagées et verbales.</s> d\'analyse syntaxique. Cependant, peu d\'entre elles reposent sur un véritable formalisme linguistique. Nous présentons ici un aperçu du logiciel DECID développé au GETA afin d\'informatiser le processus de rédaction du dictionnaire explicatif et combinatoire français contemporain.</s> dans le contexte des approches à base de grammaires faiblement sensibles au contexte, cette contribution passe en revue le problème de l\'extraction de l\'arbre d\'analyse le plus probable au sens de Data-Oriented Parsing (DOP). Une démonstration formelle de l\'utilisabilité des méthodes de Monte-Carlo est donnée, puis une technique d\'échantil

Résumé des résumés concentré, ce qui donne un petit paragraphe résumant le document.

In [35]:
summaries_for_model_summarized = []
for i in summaries_for_model:
  inputs = tokenizer.encode(i, return_tensors="pt", max_length=512)
  outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
  summaries_for_model_summarized.append(tokenizer.decode(outputs[0]))

In [36]:
summaries_for_model_summarized = [re.sub("<pad>","",i) for i in summaries_for_model_summarized]
summaries_for_model_summarized = [re.sub("<extra_id_\d>","",i) for i in summaries_for_model_summarized]
summaries_for_model_summarized

[" dans le contexte d'approches linguistiques faiblement sensibles, cette contribution passe en revue le problème de l'extraction de l'arbre d'analyse linguistique.</s>",
 " d'une méthodologie de construction et de consultation du lexique verbal. Cette communication décrit en détail un outil informatique de construction et de consultation du lexique verbal. Cette méthodologie a été développée dans le cadre de GRACE.</s>",
 " n'est pas un jeu de mots gratuit, mais bien la nécessité de soumettre à l'examen une méthode de correction linguistique plus fine que celle utilisée dans les applications classiques de CHM1.</s>",
 " linguistique, sémantique, etc.) n'a jamais fait recette auprès des littéraires, des psycho-socios, des politiques et des pouvoirs dont tout le monde a oublié sur quelles complices exactes ils se fondaient.</s>",
 ' Lexique, Lexique, Lexique, Lexique, Lexique, Lexique, Lexique, Lexique, Lexique, Lexique, Lexique, Lexique, Lexique, Lexique, Lexique, Lexique, Lexique, Lex

Ecriture des fichiers de résultats dans le stockage en ligne, pour conserver les résultats.

In [37]:
MyFile=open('T5-base_summaries.txt','w')

for element in summaries:
     MyFile.write(element)
     MyFile.write('\n')
MyFile.close()

MyFile=open('T5-base_summaries_of_model.txt','w')

for element in summaries_for_model:
     MyFile.write(element)
     MyFile.write('\n')
MyFile.close()

MyFile=open('T5-base_summaries_of_summaries.txt','w')

for element in summaries_for_model_summarized:
     MyFile.write(element)
     MyFile.write('\n')
MyFile.close()

!cp T5-base_summaries.txt "/content/drive/MyDrive/datasets/NLP/"
!cp T5-base_summaries_of_model.txt "/content/drive/MyDrive/datasets/NLP/"
!cp T5-base_summaries_of_summaries.txt "/content/drive/MyDrive/datasets/NLP/"

J'ai essayé beaucoup de méthode, et regardé et lu beaucoup de document, notamment sur huggingface et transformers.
J'ai préféré ces méthodes à celle de modèle keras avec des couches lstm car je n'étais vraiment pas convaincue par les performances de cette technique lors de la séance sur la génération de texte.
La LDA ne fournissait pas également de résultat des plus explicites et pertinents à mon gout, j'ai donc délaisser ces méthodes pour la structure encodeur décodeur fourni par transformers.
J'ai essayé d'utilisé un modèle prométeur par facebook, mais celui ne marchais pas, du fait du nom différent d'un attribut, la fonction generate ne pouvait récupérer l'encodeur du modèle ce qui provoquais une erreur arrétant l'éxecution du programme.



La méthode que j'ai ici dévellopé consiste à résumer les résumés d'article concaténé pour ensuite concaténé et résumer à nouveau ces résumés.
En ayant lû le contenu final, il touche bien aux différent principe du NLP et aborde les grands axes et les grandes méthodes de ce domaine, donnant un vue globale du contenu du corpus.